In [ ]:
/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


In [ ]:
'''/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

A1: 
SELECT name FROM `Facilities` WHERE membercost > 0.0;

/* Q2: How many facilities do not charge a fee to members? */

A2:
SELECT name FROM `Facilities` WHERE membercost = 0;

4: Badminton Court, Table Tennis, Snooker Table, Pool Table

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

A3:
SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost < 0.20 * monthlymaintenance AND membercost > 0.0;

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

A4:
SELECT * FROM `Facilities` WHERE facid IN (1,5);


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

A5:
SELECT name, monthlymaintenance, 
    (CASE WHEN monthlymaintenance > 100 THEN 'expensive' 
    ELSE 'cheap' END) AS label 
    FROM `Facilities`

name            monthlymaintenance label
Badminton Court 50                 cheap
Massage Room 1  3000               expensive
Massage Room 2  3000               expensive
Pool Table      15                 cheap
Snooker Table   15                 cheap
Squash Court    80                 cheap
Table Tennis    10                 cheap
Tennis Court 1  200                expensive
Tennis Court 2  200                expensive


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

A6:
SELECT firstname, surname FROM `Members` WHERE joindate IN(SELECT max(joindate) FROM Members);

Darren Smith

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

A7:
SELECT DISTINCT m.firstname, m.surname 
FROM `Bookings` AS b 
INNER JOIN Facilities as f ON b.facid = f.facid 
INNER JOIN Members as m ON b.memid = m.memid 
WHERE f.facid = 0 OR f.facid = 1;

firstname   surname
Tracy       Smith
GUEST       GUEST
Tim         Rownam
Darren      Smith
Janice      Joplette
Gerald      Butters
Tim         Boothe
Burton      Tracy
Nancy       Dare
Ponder      Stibbons
Charles     Owen
Anne        Baker
David       Jones
Jack        Smith
Timothy     Baker
Florence    Bader
David       Pinker
Jemima      Farrell
Ramnaresh   Sarwin
Joan        Coplin
Douglas     Jones
David       Farrell
Millicent   Purview
Henrietta   Rumney
John        Hunt


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

A8:
SELECT f.name AS facility, 
    CONCAT(m.firstname, ' ', m.surname) AS name, 
    (CASE WHEN m.memid = 0 THEN f.guestcost * b.slots 
    ELSE f.membercost * b.slots END) AS cost 
    FROM `Bookings` as b 
    INNER JOIN Facilities as f on b.facid = f.facid 
    INNER JOIN Members as m on b.memid = m.memid 
    WHERE ((f.guestcost * b.slots > 30.0 AND m.memid = 0) OR      	(f.membercost * b.slots > 30.0 AND m.memid > 0))
        AND EXTRACT(YEAR from starttime) = 2012 
        AND EXTRACT(MONTH from starttime) = 9 
        AND EXTRACT(DAY FROM starttime) = 14
ORDER BY cost DESC;

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

A9:
SELECT * FROM (
    SELECT CONCAT(m.firstname, ' ', m.surname) AS fullname,
    name AS facility,
        (CASE WHEN b.memid = 0 
        THEN (slots * guestcost) 
        ELSE (slots * membercost) END) AS cost
    FROM Bookings as b
        INNER JOIN Members as m
            ON b.memid = m.memid
        INNER JOIN Facilities as f
            ON b.facid = f.facid
    WHERE EXTRACT(YEAR from starttime) = 2012 
        AND EXTRACT(MONTH from starttime) = 9 
        AND EXTRACT(DAY FROM starttime) = 14) AS subquery
WHERE cost > 30
ORDER BY cost DESC;
''''''

In [1]:
'''example'''

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
'''Question 10'''

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT f.name,
            SUM(CASE WHEN b.memid = 0 THEN (slots * guestcost)
        ELSE (slots * membercost) END) AS revenue
        FROM Bookings as b
        INNER JOIN Members as m ON b.memid = m.memid
        INNER JOIN Facilities as f on b.facid = f.facid
        GROUP BY f.name HAVING revenue < 1000;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


In [11]:
'''Question 11'''

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT m1.firstname || ' ' || m1.surname, m2.firstname || ' ' || m2.surname AS recommend
        FROM Members as m1
        INNER JOIN Members as m2
        ON m1.recommendedby = m2.memid
        ORDER BY m1.surname, m1.firstname
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Florence Bader', 'Ponder Stibbons')
('Anne Baker', 'Ponder Stibbons')
('Timothy Baker', 'Jemima Farrell')
('Tim Boothe', 'Tim Rownam')
('Gerald Butters', 'Darren Smith')
('Joan Coplin', 'Timothy Baker')
('Erica Crumpet', 'Tracy Smith')
('Nancy Dare', 'Janice Joplette')
('Matthew Genting', 'Gerald Butters')
('John Hunt', 'Millicent Purview')
('David Jones', 'Janice Joplette')
('Douglas Jones', 'David Jones')
('Janice Joplette', 'Darren Smith')
('Anna Mackenzie', 'Darren Smith')
('Charles Owen', 'Darren Smith')
('David Pinker', 'Jemima Farrell')
('Millicent Purview', 'Tracy Smith')
('Henrietta Rumney', 'Matthew Genting')
('Ramnaresh Sarwin', 'Florence Bader')
('Jack Smith', 'Darren Smith')
('Ponder Stibbons', 'Burton Tracy')
('Henry Worthington-Smyth', 'Tracy Smith')


In [15]:
'''Question 12'''

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT f.name, COUNT(*)
        FROM Bookings as b
        INNER JOIN Members as m ON b.memid = m.memid
        INNER JOIN Facilities as f on b.facid = f.facid
        WHERE b.memid != 0
        GROUP BY f.name;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', 344)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Pool Table', 783)
('Snooker Table', 421)
('Squash Court', 195)
('Table Tennis', 385)
('Tennis Court 1', 308)
('Tennis Court 2', 276)


In [22]:
'''Question 13'''

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT f.name, strftime('%m', starttime), COUNT(*)
        FROM Bookings as b
        INNER JOIN Members as m ON b.memid = m.memid
        INNER JOIN Facilities as f on b.facid = f.facid
        WHERE b.memid != 0
        GROUP BY f.name, strftime('%m', starttime);
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', '07', 51)
('Badminton Court', '08', 132)
('Badminton Court', '09', 161)
('Massage Room 1', '07', 77)
('Massage Room 1', '08', 153)
('Massage Room 1', '09', 191)
('Massage Room 2', '07', 4)
('Massage Room 2', '08', 9)
('Massage Room 2', '09', 14)
('Pool Table', '07', 103)
('Pool Table', '08', 272)
('Pool Table', '09', 408)
('Snooker Table', '07', 68)
('Snooker Table', '08', 154)
('Snooker Table', '09', 199)
('Squash Court', '07', 23)
('Squash Court', '08', 85)
('Squash Court', '09', 87)
('Table Tennis', '07', 48)
('Table Tennis', '08', 143)
('Table Tennis', '09', 194)
('Tennis Court 1', '07', 65)
('Tennis Court 1', '08', 111)
('Tennis Court 1', '09', 132)
('Tennis Court 2', '07', 41)
('Tennis Court 2', '08', 109)
('Tennis Court 2', '09', 126)
